In [1]:
'''
    20251201:
        开放问题回答，生成--for 评测

    20251208:
        生成judge的文件
    20251209:
        修改metric
    20251210:
        llama2 base model generation
'''

'\n    20251201:\n        开放问题回答，生成--for 评测\n\n    20251208:\n        生成judge的文件\n    20251209:\n        修改metric\n    20251210:\n        llama2 base model generation\n'

In [28]:
import numpy as np

In [4]:
import datasets

In [5]:
import transformers

import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
# download checkpoint
from accelerate import load_checkpoint_and_dispatch
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To prevent long warnings :)

#from accelerate import load_checkpoint_and_dispatch

from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

#import util
import importlib

#importlib.reload(util)      # 只能 reload 模块本身
#from util import calc_ppl, get_test_data   # reload 后再重新 import 函数

import pandas as pd
import copy

In [6]:
def load_model(checkpoint):
    config = AutoConfig.from_pretrained(checkpoint,trust_remote_code=True)
    print('checkpoint:', checkpoint)
    
    
    if True:
    
        device_map='cuda'
        model= AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, torch_dtype= torch.bfloat16,device_map=device_map ) # for download model weight
    
    else:
        with init_empty_weights():
            model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
        model = load_checkpoint_and_dispatch(
            model, checkpoint, device_map="auto", dtype=torch.bfloat16#, no_split_module_classes=["GPTJBlock"]
        )
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)#AutoTokenizer.from_pretrained("/home/work/lyftri/projects/model_zoo/compass_sea_13b_s4_merge2HF_org_convert_TP_1_PP_2",  trust_remote_code=True)#AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

    return model, tokenizer

In [7]:
'''
load test set

'''

'\nload test set\n\n'

In [8]:
ds_test = datasets.load_dataset('json', data_files ='/root/autodl-fs/LRP/open_ended_dataset/all_data.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
ds_test['train'][121]

{'text': 'Điều gì sẽ xảy ra nếu người Aztec đã thành công trong việc đẩy lùi các chinh phục viên Tây Ban Nha?',
 'lang': 'vi',
 '__index_level_0__': 51}

# 20251208

In [110]:
ds_lape = datasets.load_dataset('json', data_files='/root/autodl-fs/LRP/open_ended_data_generation/20251201_LAPE_org_generation.json')

In [111]:
df_lape = ds_lape['train'].to_pandas().loc[df_lape['model_type']!='org_model|open_ended'].reset_index(drop=True)

In [112]:
df_lape.head()

,text,lang,__index_level_0__,answer,model_type
0,How can I improve my time management skills?,en,0,Time management is the process of planning and...,mask_en|open_ended
1,What are the most effective ways to deal with ...,en,1,"Stress is a normal part of life, but chronic s...",mask_en|open_ended
2,What are the main differences between Python a...,en,2,Python and JavaScript are two popular programm...,mask_en|open_ended
3,How can I increase my productivity while worki...,en,3,Here are some tips to help you increase your p...,mask_en|open_ended
4,Can you explain the basics of quantum computing?,en,4,Quantum computing is a rapidly growing field t...,mask_en|open_ended


In [113]:
df_lape['model_type'].value_counts()

model_type
mask_en|open_ended    210
mask_vi|open_ended    210
mask_zh|open_ended    210
Name: count, dtype: int64

In [114]:
# new 

In [115]:
ds_lrp = datasets.load_dataset('json', data_files='/root/autodl-fs/LRP/open_ended_data_generation/20251204_all_exp/1208_all.json')

In [116]:
df_lrp = ds_lrp['train'].to_pandas()

In [117]:
ds_lrp

DatasetDict({
    train: Dataset({
        features: ['text', 'lang', '__index_level_0__', 'answer', 'model_type'],
        num_rows: 48930
    })
})

In [118]:
df_all = pd.concat([df_lape, df_lrp]).drop_duplicates(subset=['text', 'lang', 'answer', 'model_type']).reset_index(drop=True)

In [119]:
df_all['model_type'].value_counts()

model_type
org_model|open_ended                                       210
mask_en|open_ended                                         210
gap_rate_top_0_5perc_LRP_kur_res_zscore_zh|open_ended      210
gap_rate_top_1perc_LRP_kur_res_en|open_ended               210
gap_rate_top_1perc_LRP_kur_res_vi|open_ended               210
                                                          ... 
gaprate_th_11_selected_LRP_kur_res_vi_zscore|open_ended    210
gaprate_th_11_selected_LRP_kur_res_zh_zscore|open_ended    210
gaprate_th_14_selected_LRP_kur_res_en_zscore|open_ended    210
gaprate_th_14_selected_LRP_kur_res_vi_zscore|open_ended    210
gaprate_th_14_selected_LRP_kur_res_zh_zscore|open_ended    210
Name: count, Length: 154, dtype: int64

In [123]:
len(df_all)/210

154.0

In [124]:
'''
生成judge的prompt

'''
def generate_judge_file(df, save_path):
    prompt_template = (
        "You are a neutral judge. Score the model’s answer from 1 to 10 based on correctness, completeness, clarity, and usefulness.\n"
        "Respond ONLY in this JSON format:\n"
        '{{"score": <1-10>, "reason": "<brief reason>"}}\n\n'
        "Question:\n{question}\n\n"
        "Answer:\n{answer}"
    )
    
    def get_judge_promt(x):
        return prompt_template.format(question= x[0], answer= x[1])
    df['judge_prompt'] = df[['text','answer']].apply(lambda x: prompt_template.format(question= x[0], answer= x[1]), axis=1)
    ds_final = datasets.Dataset.from_pandas(df)
    
    #ds_final.to_json('~/autodl-fs/LRP/open_ended_data_generation/20251201_LRP_based_generation4judge.json',index=False)
    ds_final.to_json(save_path, index=False)


In [125]:
generate_judge_file(df_all, '/root/autodl-fs/LRP/open_ended_data_generation/20251204_all_exp/1208_all_generation4judge.json')

/tmp/ipykernel_2146/483060424.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['judge_prompt'] = df[['text','answer']].apply(lambda x: prompt_template.format(question= x[0], answer= x[1]), axis=1)


Creating json from Arrow format:   0%|          | 0/33 [00:00<?, ?ba/s]

In [23]:
# 分析结果

In [11]:
judege_path ='/root/autodl-fs/LRP/open_ended_data_generation/20251204_all_exp/1208_all_generation4judge.json_gpt4o.json'

In [12]:
ds_judge_res = datasets.load_dataset('json', data_files = judege_path)




In [13]:
ds_judge_res['train'][0]

{'text': 'How can I improve my time management skills?',
 'lang': 'en',
 '__index_level_0__': 0,
 'answer': "Time management is the process of planning and controlling the amount of time spent on different activities to maximize efficiency and productivity. Here are some tips that can help you improve your time management skills:\n\n1. Set clear goals: Identify what you want to achieve and set specific, measurable, achievable, relevant, and time-bound (SMART) goals. This will help you prioritize your tasks and focus on what's important.\n2. Create a schedule: Plan out your day, week, or month in advance using a calendar or planner. Block out dedicated time for work, breaks, and leisure activities. Be sure to include time for unexpected interruptions and flexibility.\n3. Prioritize tasks: Determine which tasks are most important and need to be completed first. Use the Eisenhower Matrix to categorize tasks into urgent vs. important, and focus on the most critical ones first.\n4. Avoid mu

In [14]:
def get_analysis_res(ds_judge_res):
    df_judge_res = ds_judge_res['train'].to_pandas()
    # get score
    def get_score(x):
        try:
            return  json.loads(x)['score']
        except:
            return -1
    
    df_judge_res['score'] = df_judge_res['output'].apply(get_score)

    df_judge_res['class_tpye'] = df_judge_res['lang']+'|' + df_judge_res['model_type']

    result_dict = {}
    for imodel_type in list(df_judge_res['class_tpye'].value_counts().keys()):
        print('='*20)
        print('imodel_type:', imodel_type)
        df_tmp = df_judge_res.loc[df_judge_res['class_tpye'] == imodel_type].reset_index(drop=True)
        df_tmp = df_tmp.loc[df_tmp['score']!=-1].reset_index(drop=True)
        print('valid count:', len(df_tmp))
        result_dict[imodel_type] = df_tmp['score'].mean()
              
    return result_dict, df_judge_res


In [15]:
result_dict, df_judge_res = get_analysis_res(ds_judge_res)

imodel_type: zh|org_model|open_ended
valid count: 70
imodel_type: en|mask_en|open_ended
valid count: 70
imodel_type: vi|mask_en|open_ended
valid count: 70
imodel_type: zh|mask_en|open_ended
valid count: 70
imodel_type: en|mask_vi|open_ended
valid count: 68
imodel_type: vi|mask_vi|open_ended
valid count: 70
imodel_type: zh|mask_vi|open_ended
valid count: 70
imodel_type: en|mask_zh|open_ended
valid count: 70
imodel_type: vi|mask_zh|open_ended
valid count: 69
imodel_type: zh|mask_zh|open_ended
valid count: 70
imodel_type: en|gaprate_th_0.9_selected_LRP_kur_res_zh|open_ended
valid count: 70
imodel_type: vi|gaprate_th_0.9_selected_LRP_kur_res_zh|open_ended
valid count: 70
imodel_type: zh|gaprate_th_0.9_selected_LRP_kur_res_zh|open_ended
valid count: 70
imodel_type: en|gaprate_th_11_selected_LRP_kur_res_en_zscore|open_ended
valid count: 70
imodel_type: zh|gap_rate_top_1perc_LRP_kur_res_vi|open_ended
valid count: 70
imodel_type: en|gap_rate_top_1perc_LRP_kur_res_zh|open_ended
valid count: 69


In [16]:
df_judge_res.head()

,text,lang,__index_level_0__,answer,model_type,judge_prompt,output,score,class_tpye
0,How can I improve my time management skills?,en,0,Time management is the process of planning and...,mask_en|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 10, ""reason"": ""The answer is correct...",10,en|mask_en|open_ended
1,What are the most effective ways to deal with ...,en,1,"Stress is a normal part of life, but chronic s...",mask_en|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 9, ""reason"": ""The answer is comprehe...",9,en|mask_en|open_ended
2,What are the main differences between Python a...,en,2,Python and JavaScript are two popular programm...,mask_en|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 6, ""reason"": ""The answer contains se...",6,en|mask_en|open_ended
3,How can I increase my productivity while worki...,en,3,Here are some tips to help you increase your p...,mask_en|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 9, ""reason"": ""The answer is correct,...",9,en|mask_en|open_ended
4,Can you explain the basics of quantum computing?,en,4,Quantum computing is a rapidly growing field t...,mask_en|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 9, ""reason"": ""The answer provides a ...",9,en|mask_en|open_ended


In [17]:
len(result_dict)

462

In [18]:
462/3

154.0

In [19]:

def get_mask_neuron_lang(class_type):
    tmp_list = class_type.split('|')
    if tmp_list[1]=='org_model':
        return 'no_mask'
    
    mid_name_list = tmp_list[1].split('_')
    count = int('en' in mid_name_list) + int('vi' in mid_name_list) + int('zh' in mid_name_list) 
    assert count == 1,(class_type, count)
    if 'en' in mid_name_list:
        mid_name_list.remove('en')
        return 'en'
    if 'vi' in mid_name_list:
        mid_name_list.remove('vi')
        return 'vi'
    if 'zh' in mid_name_list:
        mid_name_list.remove('zh')
        return 'zh'

        

def get_method_name(class_type):
    tmp_list = class_type.split('|')
    if tmp_list[1]=='org_model':
        return 'no_mask'
        
    mid_name_list = tmp_list[1].split('_')
    count = int('en' in mid_name_list) + int('vi' in mid_name_list) + int('zh' in mid_name_list) 
    
    if 'en' in mid_name_list:
        mid_name_list.remove('en')
    if 'vi' in mid_name_list:
        mid_name_list.remove('vi')
    if 'zh' in mid_name_list:
        mid_name_list.remove('zh')

    return '_'.join(mid_name_list)


In [20]:
df_judge_res['mask_neuron_lang'] = df_judge_res['class_tpye'].apply(get_mask_neuron_lang)

In [21]:
df_judge_res['method_name'] = df_judge_res['class_tpye'].apply(get_method_name)

In [22]:
df_judge_res['method_name'].value_counts()

method_name
mask                                                                       630
gaprate_th_0.9_selected_LRP_kur_res                                        630
gaprate_th_11_selected_LRP_kur_res_zscore                                  630
gaprate_th_14_selected_LRP_kur_res_zscore                                  630
gaprate_th_42_selected_LRP_kur_res_zscore                                  630
gaprate_th_70_selected_LRP_kur_res_zscore                                  630
soft_preference_th_0.5_selected_LRP_kur_res_Not_zscore_margin_selected     630
soft_preference_th_0.5_selected_LRP_kur_res_zscore_margin_selected         630
soft_preference_th_0.95_selected_LRP_kur_res_Not_zscore_margin_selected    630
soft_preference_th_0.95_selected_LRP_kur_res_zscore_margin_selected        630
soft_preference_th_0.99_selected_LRP_kur_res_Not_zscore_margin_selected    630
soft_preference_th_0.99_selected_LRP_kur_res_zscore_margin_selected        630
soft_preference_th_0.9_selected_LRP_kur_

In [23]:
df_judge_res.head()

,text,lang,__index_level_0__,answer,model_type,judge_prompt,output,score,class_tpye,mask_neuron_lang,method_name
0,How can I improve my time management skills?,en,0,Time management is the process of planning and...,mask_en|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 10, ""reason"": ""The answer is correct...",10,en|mask_en|open_ended,en,mask
1,What are the most effective ways to deal with ...,en,1,"Stress is a normal part of life, but chronic s...",mask_en|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 9, ""reason"": ""The answer is comprehe...",9,en|mask_en|open_ended,en,mask
2,What are the main differences between Python a...,en,2,Python and JavaScript are two popular programm...,mask_en|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 6, ""reason"": ""The answer contains se...",6,en|mask_en|open_ended,en,mask
3,How can I increase my productivity while worki...,en,3,Here are some tips to help you increase your p...,mask_en|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 9, ""reason"": ""The answer is correct,...",9,en|mask_en|open_ended,en,mask
4,Can you explain the basics of quantum computing?,en,4,Quantum computing is a rapidly growing field t...,mask_en|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 9, ""reason"": ""The answer provides a ...",9,en|mask_en|open_ended,en,mask


In [24]:
df_judge_res_no_mask = df_judge_res.loc[df_judge_res['method_name'] == 'no_mask'].reset_index(drop=True)

In [25]:
len(df_judge_res_no_mask)

210

In [26]:
df_judge_res_no_mask.head(2)

,text,lang,__index_level_0__,answer,model_type,judge_prompt,output,score,class_tpye,mask_neuron_lang,method_name
0,How can I improve my time management skills?,en,0,Time management is the process of planning and...,org_model|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 10, ""reason"": ""The answer is correct...",10,en|org_model|open_ended,no_mask,no_mask
1,What are the most effective ways to deal with ...,en,1,"Stress is a normal part of life, but chronic s...",org_model|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 9, ""reason"": ""The answer is comprehe...",9,en|org_model|open_ended,no_mask,no_mask


In [96]:
'''
    calc LPS:
        LPS = drop(target) / mean(drop(other)) * mean(drop)

'''

# get org
df_judge_res['specific_method'] = df_judge_res['class_tpye'].apply(lambda x: x.split('|')[1])
df_org = df_judge_res.loc[df_judge_res['specific_method'] =='org_model' ]

# 获取不同语种测试集合的分数
def get_all_lang_score(df_org):
    lang_list = ['en', 'vi', 'zh']
    lang_score_baseline = {}
    for ilang in lang_list:
        lang_score_baseline[ilang] = df_org.loc[df_org['lang']==ilang]['score'].mean()
    return lang_score_baseline
# baseline
lang_score_baseline = get_all_lang_score(df_org)

# calc LPS:
res_score = {}
res_acuall_score = {}
for iexp_name in result_dict.keys():
    specific_method = iexp_name.split('|')[1]
    if specific_method=='org_model':continue

    df_tmp = df_judge_res.loc[df_judge_res['specific_method']==specific_method ].reset_index(drop=True)
    lang_score_tmp = get_all_lang_score(df_tmp)

    assert len(dict(df_tmp['mask_neuron_lang'].value_counts()))==1 and len(dict(df_tmp['method_name'].value_counts()))==1

   

    assert df_tmp['mask_neuron_lang'][0] not in res_score,(specific_method, df_tmp['mask_neuron_lang'][0], res_score)

    if df_tmp['method_name'][0] not in res_score:
        res_score[df_tmp['method_name'][0]] = {}
        res_acuall_score[df_tmp['method_name'][0]] = {}
         
    # calc per lang score
    tmp_drop = {}
    for ikey in lang_score_baseline.keys():
        tmp_drop[ikey] = lang_score_baseline[ikey] - lang_score_tmp[ikey]

    tmp_taget_lang = df_tmp['mask_neuron_lang'][0]
    lang_list = ['en', 'vi', 'zh']
    lang_list.remove(tmp_taget_lang)

    # LPS
    #LPS = drop(target) / mean(drop(other)) * mean(drop)
    #LPS = tmp_drop[tmp_taget_lang]/((tmp_drop[lang_list[0]] + tmp_drop[lang_list[1]])/2)*(sum(tmp_drop.values())/3)
    #res_score[df_tmp['method_name'][0]][tmp_taget_lang] = LPS

    # sigmoid LPS
    k = 2
    f = lambda x: 2/(1+np.exp(-k*x))
    # sigmoid(drop_target) /sigmoid(mean_drop_other) * sigmoid(all_mean_drop) 
    
    #LPS_sig = f(tmp_drop[tmp_taget_lang]) / f(((tmp_drop[lang_list[0]] + tmp_drop[lang_list[1]])/2)) #* f(sum(tmp_drop.values())/3)
    
    #res_score[df_tmp['method_name'][0]][tmp_taget_lang] = LPS_sig

    #res_acuall_score[df_tmp['method_name'][0]][tmp_taget_lang] = lang_score_tmp


    # only two part 
    #sigmoid(drop_target - mean_drop_other) 
    # 3 2 =1
    # -3 -4 =1
    res_score[df_tmp['method_name'][0]][tmp_taget_lang] = f((tmp_drop[tmp_taget_lang]) -((tmp_drop[lang_list[0]] + tmp_drop[lang_list[1]])/2))
    res_acuall_score[df_tmp['method_name'][0]][tmp_taget_lang] =  lang_score_tmp

    if False:
        #LPS★ = (drop(target) - mean(drop(other))) * drop(target)
        drop_target = tmp_drop[tmp_taget_lang]
        mean_drop_other = ((tmp_drop[lang_list[0]] + tmp_drop[lang_list[1]])/2)
        LPS_star = (drop_target - mean_drop_other) * drop_target
        res_score[df_tmp['method_name'][0]][tmp_taget_lang] = LPS_star
    

    
    

    
    

    
    

    

    






In [97]:
lang_score_baseline

{'en': np.float64(7.9),
 'vi': np.float64(6.985714285714286),
 'zh': np.float64(6.814285714285714)}

In [98]:
lang_score_baseline

{'en': np.float64(7.9),
 'vi': np.float64(6.985714285714286),
 'zh': np.float64(6.814285714285714)}

In [99]:
# res_score_mean

final_score={}
for imethod in res_score:
    final_score[imethod] = sum(res_score[imethod].values())/len(res_score[imethod].values())

In [100]:
sorted(final_score.items(), key = lambda x: x[1], reverse=True )

[('soft_preference_th_0.5_selected_LRP_kur_res_Not_zscore_margin_selected',
  np.float64(1.4282313135712563)),
 ('gap_rate_bottom_0_5perc_LRP_kur_res_zscore',
  np.float64(1.3558690898528127)),
 ('gaprate_th_0.9_selected_LRP_kur_res', np.float64(1.3358791945003015)),
 ('soft_preference_th_0.5_selected_LRP_kur_res_zscore_margin_selected',
  np.float64(1.326185445149598)),
 ('gap_rate_bottom_1perc_LRP_kur_res_zscore', np.float64(1.224075792743788)),
 ('th_0_selected_LRP_kur_res_zscore', np.float64(1.2225990308674604)),
 ('top_0_5perc_LRP_kur_res', np.float64(1.1730862634608876)),
 ('th_1_selected_LRP_kur_res_zscore', np.float64(1.166688385112652)),
 ('gap_rate_top_0_0_1perc_LRP_kur_res', np.float64(1.129726167124258)),
 ('gap_rate_bottom_0_1_perc_LRP_kur_res_zscore',
  np.float64(1.1212128780938153)),
 ('soft_preference_th_0.99_selected_LRP_kur_res_Not_zscore_margin_selected',
  np.float64(1.1157711609784287)),
 ('gap_rate_top_0_5perc_LRP_kur_res', np.float64(1.1080744997968346)),
 ('gap

In [101]:
lang_score_baseline

{'en': np.float64(7.9),
 'vi': np.float64(6.985714285714286),
 'zh': np.float64(6.814285714285714)}

In [93]:
res_acuall_score['mask']

{'en': {'en': np.float64(7.942857142857143),
  'vi': np.float64(7.128571428571429),
  'zh': np.float64(6.957142857142857)},
 'vi': {'en': np.float64(7.685714285714286),
  'vi': np.float64(7.3),
  'zh': np.float64(6.9714285714285715)},
 'zh': {'en': np.float64(8.028571428571428),
  'vi': np.float64(7.185714285714286),
  'zh': np.float64(6.728571428571429)}}

In [95]:
res_acuall_score['th_1_selected_LRP_kur_res_zscore']

{'zh': {'en': np.float64(6.914285714285715),
  'vi': np.float64(3.6857142857142855),
  'zh': np.float64(4.071428571428571)},
 'en': {'en': np.float64(6.0),
  'vi': np.float64(3.357142857142857),
  'zh': np.float64(3.9714285714285715)},
 'vi': {'en': np.float64(7.9714285714285715),
  'vi': np.float64(5.457142857142857),
  'zh': np.float64(6.085714285714285)}}

In [34]:
get_all_lang_score(df_judge_res.loc[df_judge_res['method_name'] =='th_1_selected_LRP_kur_res_zscore' ])

{'en': np.float64(6.961904761904762),
 'vi': np.float64(4.166666666666667),
 'zh': np.float64(4.70952380952381)}

In [35]:
df_judge_res.loc[df_judge_res['method_name'] =='th_1_selected_LRP_kur_res_zscore' ].head()

,text,lang,__index_level_0__,answer,model_type,judge_prompt,output,score,class_tpye,mask_neuron_lang,method_name,specific_method
10920,How can I improve my time management skills?,en,0,Unterscheidung between important and unimporta...,th_1_selected_LRP_kur_res_en_zscore|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 9, ""reason"": ""The answer is correct,...",9,en|th_1_selected_LRP_kur_res_en_zscore|open_ended,en,th_1_selected_LRP_kur_res_zscore,th_1_selected_LRP_kur_res_en_zscore
10921,What are the most effective ways to deal with ...,en,1,Unterscheidung between healthy and unhealthy c...,th_1_selected_LRP_kur_res_en_zscore|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 5, ""reason"": ""The answer provides a ...",5,en|th_1_selected_LRP_kur_res_en_zscore|open_ended,en,th_1_selected_LRP_kur_res_zscore,th_1_selected_LRP_kur_res_en_zscore
10922,What are the main differences between Python a...,en,2,Unterscheidung between Python and JavaScript i...,th_1_selected_LRP_kur_res_en_zscore|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 4, ""reason"": ""The answer contains se...",4,en|th_1_selected_LRP_kur_res_en_zscore|open_ended,en,th_1_selected_LRP_kur_res_zscore,th_1_selected_LRP_kur_res_en_zscore
10923,How can I increase my productivity while worki...,en,3,Unterscheidung between productive and unproduc...,th_1_selected_LRP_kur_res_en_zscore|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 9, ""reason"": ""The answer is comprehe...",9,en|th_1_selected_LRP_kur_res_en_zscore|open_ended,en,th_1_selected_LRP_kur_res_zscore,th_1_selected_LRP_kur_res_en_zscore
10924,Can you explain the basics of quantum computing?,en,4,Unterscheidung between classical and quantum c...,th_1_selected_LRP_kur_res_en_zscore|open_ended,You are a neutral judge. Score the model’s ans...,"{""score"": 8, ""reason"": ""The answer provides a ...",8,en|th_1_selected_LRP_kur_res_en_zscore|open_ended,en,th_1_selected_LRP_kur_res_zscore,th_1_selected_LRP_kur_res_en_zscore


In [103]:
result_dict['en|mask_en|open_ended']

np.float64(7.942857142857143)

In [107]:
result_dict['vi|mask_en|open_ended']

np.float64(7.128571428571429)

In [108]:
result_dict['zh|mask_en|open_ended']

np.float64(6.957142857142857)

In [104]:
result_dict['en|org_model|open_ended']

np.float64(8.028985507246377)

In [105]:
result_dict['vi|org_model|open_ended']

np.float64(6.985714285714286)

In [106]:
result_dict['zh|org_model|open_ended']

np.float64(6.814285714285714)

In [22]:
result_dict

{'zh|org_model|open_ended': np.float64(6.814285714285714),
 'en|mask_en|open_ended': np.float64(7.942857142857143),
 'vi|mask_en|open_ended': np.float64(7.128571428571429),
 'zh|mask_en|open_ended': np.float64(6.957142857142857),
 'en|mask_vi|open_ended': np.float64(7.9411764705882355),
 'vi|mask_vi|open_ended': np.float64(7.3),
 'zh|mask_vi|open_ended': np.float64(6.9714285714285715),
 'en|mask_zh|open_ended': np.float64(8.028571428571428),
 'vi|mask_zh|open_ended': np.float64(7.304347826086956),
 'zh|mask_zh|open_ended': np.float64(6.728571428571429),
 'en|gaprate_th_0.9_selected_LRP_kur_res_zh|open_ended': np.float64(1.0),
 'vi|gaprate_th_0.9_selected_LRP_kur_res_zh|open_ended': np.float64(1.0),
 'zh|gaprate_th_0.9_selected_LRP_kur_res_zh|open_ended': np.float64(1.0),
 'en|gaprate_th_11_selected_LRP_kur_res_en_zscore|open_ended': np.float64(6.957142857142857),
 'zh|gap_rate_top_1perc_LRP_kur_res_vi|open_ended': np.float64(6.557142857142857),
 'en|gap_rate_top_1perc_LRP_kur_res_zh|op